# HW 3

### Problem 1 (1 points)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1. Implement function to generate common secret key within multiplicative group of given Finite field with known generator

*Note. You can assume that all the numbers are small, for example, less than $2^{16}$.*

In [1]:
from random import randint

def generate_common_key(p,g, private_keys=None):
    # Generate random private keys for each party
    if private_keys is None:
        private_key_A = randint(1, p-1)
        private_key_B = randint(1, p-1)
    else:
        private_key_A = private_keys['A']
        private_key_B = private_keys['B']

    # Calculate public keys for each party
    public_key_A = pow(g, private_key_A, p)
    public_key_B = pow(g, private_key_B, p)

    # Calculate the common secret key using the public keys
    common_secret_key_A = pow(public_key_B, private_key_A, p)
    common_secret_key_B = pow(public_key_A, private_key_B, p)

    # Check if the common secret keys match
    assert common_secret_key_A == common_secret_key_B

    return common_secret_key_A

generate_common_key(17,11)


4

2. Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [2]:
private_keys = {'A':7,
                'B':11}

generate_common_key(17,11,private_keys)

7

### Problem 2 (2 points)

We will implement an example which demonstrates RSA public-key cryptography in an easy-to-follow manner. It should work on integers alone, and use small numbers for the sake of clarity. 

First we pick our primes. These will determine our keys.
We pick P,Q,and E such that:

    1. P and Q are prime; picked at random.
    2. 1 < E < (P-1)*(Q-1) and E is co-prime with (P-1)*(Q-1)

In [1]:
P = 97    # First prime
Q = 83    # Second prime
E = 53    # Open exponent. usually a constant; 0x10001 is common, prime is best

1. Implement complementary functions  (1 point):
    - brute-force primality test,
    - extended euclidean algorithm,
    - function for finding multiplicative inverse of x (mod y)

In [2]:
# Brute-force (i.e. try every possibility) primality test.
def isPrime(x):
    for f in range(2,x-1):
        if x%f == 0:
            return False
    return True

# Extended_Euclidean_algorithm
def eea(a,b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = eea(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

# Find the multiplicative inverse of x (mod y)
# You should use eea() function here
def find_inverse(x,y):
    gcd, x_inv, _ = eea(x, y)
    if gcd == 1:
        return x_inv % y
    else:
        raise ValueError("Multiplicative inverse does not exist")

Let's make sure the numbers we picked above are valid

In [3]:
if not isPrime(P): raise Exception("P (%i) is not prime" % (P,))
if not isPrime(Q): raise Exception("Q (%i) is not prime" % (Q,))

T=(P-1)*(Q-1) # Euler's totient (intermediate result)
# Assuming E is prime, we just have to check against T
if E<1 or E > T: raise Exception("E must be > 1 and < T")
if T%E==0: raise Exception("E is not coprime with T")

2. Find private and public keys and perform encryption and decryption (1 point)

In [4]:
D = find_inverse(E,T)
D, E, T

(2525, 53, 7872)

In [7]:
D*E/T

17.000127032520325

In [28]:
message = int(1111)
print("Initial message: %i" % message)


key = (E, P*Q)
encrypted_message = pow(message, *key)
print("Encrypted_message: %i" % encrypted_message)

key = (D, P*Q)
decrypted_message = pow(encrypted_message, *key)
print("Decrypted_message: %i" % decrypted_message)

Initial message: 1111
Encrypted_message: 4293
Decrypted_message: 1111


### Problem 3 (4 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write python function that returns all group elements of a certain elliptic curve over a finite field 

1. Write a python function that list all points of elliptic curve $y^2=x^3-10x-6$ over $F_{13}$ (2 points)

In [81]:

def elliptic_curve_points_over_finite_field(a, b, p):
    points = []
    for x in range(p):
        y_squared = (x**3 + a*x + b) % p
        for y in range(p):
            if (y*y) % p == y_squared:
                points.append((x, y))
                if y != 0:
                    points.append((x, p - y))
    return points

a = -10
b = -6
p = 13
points = elliptic_curve_points_over_finite_field(a, b, p)
print(points)

[(3, 2), (3, 11), (3, 11), (3, 2), (5, 2), (5, 11), (5, 11), (5, 2), (8, 6), (8, 7), (8, 7), (8, 6), (9, 3), (9, 10), (9, 10), (9, 3), (10, 6), (10, 7), (10, 7), (10, 6), (12, 4), (12, 9), (12, 9), (12, 4)]


2. Compute the sum of 2 different points in group above and make sure that result lies within the same group (1 point)

In [94]:
def point_addition(p1, p2, a, p):
    if p1 == "O":
        return p2
    if p2 == "O":
        return p1

    x1, y1 = p1
    x2, y2 = p2

    if p1 != p2:
        m = ((y2 - y1) * pow(x2 - x1, -1, p)) % p
    else:
        m = ((3*x1**2 + a) * pow(2*y1, -1, p)) % p

    x3 = (m**2 - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p

    return (x3, y3)

p1 = points[0]
p2 = points[4]

result = point_addition(p1, p2, a, p)

assert (result[1]**2)%13 == (result[0]**3-10*result[0]-6)%13
print("Result of point addition:", result)


Result of point addition: (5, 11)


3. Compute the sum of 2 equal points in group above and make sure that result lies within the same group (1 point)

In [95]:
p1 = points[0]
p2 = points[0]

result = point_addition(p1, p2, a, p)
assert (result[1]**2)%13 == (result[0]**3-10*result[0]-6)%13
print("Result of point addition with equal points:", result)

Result of point addition with equal points: (8, 6)


### Problem 4 (2 points)

Let us consider the finite field $\mathbb{Z}_p$ with $p=11$. A secret was shared using $(n,3)$-threshold Shamir secret sharing scheme. $4$ shares are known: $(2;9), (3;9), (5;5), (7;3)$. Reconstruct the polynomial $f(x)$ and the common secret $f(0)$.

 Note that each share is given in the form $(x_i; f(x_i))$.

To reconstruct the polynomial $f(x)$ and the common secret $f(0)$ using the given shares in the $(n,3)$-threshold Shamir secret sharing scheme, we need to perform Lagrange interpolation. The Lagrange interpolation formula is given by:

$$f(x) = \sum_{i=1}^{n} f(x_i) \cdot l_i(x)$$

where $l_i(x) = \prod_{j=1,j\neq i}^{n} \frac{x-x_j}{x_i-x_j}$.

Given the shares $(2;9), (3;9), (5;5), (7;3)$, we can construct the Lagrange basis polynomials and then compute the polynomial $f(x)$.

Let's perform the calculations:

1. Calculate the Lagrange basis polynomials:
   - For $i=1$: $l_1(x) = \frac{(x-3)(x-5)(x-7)}{(2-3)(2-5)(2-7)} = \frac{(x-3)(x-5)(x-7)}{10}$
   - For $i=2$: $l_2(x) = \frac{(x-2)(x-5)(x-7)}{(3-2)(3-5)(3-7)} = \frac{(x-2)(x-5)(x-7)}{20}$
   - For $i=3$: $l_3(x) = \frac{(x-2)(x-3)(x-7)}{(5-2)(5-3)(5-7)} = \frac{(x-2)(x-3)(x-7)}{-4}$
   - For $i=4$: $l_4(x) = \frac{(x-2)(x-3)(x-5)}{(7-2)(7-3)(7-5)} = \frac{(x-2)(x-3)(x-5)}{10}$

2. Compute the polynomial $f(x)$:
   $$f(x) = 9 \cdot l_1(x) + 9 \cdot l_2(x) + 5 \cdot l_3(x) + 3 \cdot l_4(x)$$


In order to calculate coefficients and find common secret let's write Python code:



In [98]:
from sympy import symbols, simplify

# Given shares
shares = [(2, 9), (3, 9), (5, 5), (7, 3)]

# Lagrange basis polynomials
def lagrange_basis(i, x, shares):
    result = 1
    xi, _ = shares[i]
    for j in range(len(shares)):
        if j != i:
            xj, _ = shares[j]
            result *= (x - xj) / (xi - xj)
    return result

# Compute the Lagrange interpolation
x = symbols('x')
f_x = 0
for i in range(len(shares)):
    xi, yi = shares[i]
    f_x += yi * lagrange_basis(i, x, shares)

f_x = simplify(f_x)

# Evaluate f(0) to find the common secret
common_secret = f_x.subs(x, 0)

print("The reconstructed polynomial f(x) is:", f_x)
print("The common secret f(0) is:", common_secret)


The reconstructed polynomial f(x) is: 11*x**3/60 - 5*x**2/2 + 541*x/60 - 1/2
The common secret f(0) is: -1/2
